# REST API Search Endpoint

Método de consulta de tweets históricos (hasta una semana) basándose en varios criterios, es muy adecuado para un análisis estático.

## Carga inicial de datos

In [1]:
import requests
from requests_oauthlib import OAuth1

ModuleNotFoundError: No module named 'requests_oauthlib'

Parte en la que se importa las librerías necesarias.

In [ ]:
q = 'premier league -filter:retweets AND -filter:replies'
url = 'https://api.Twitter.com/1.1/search/tweets.json'
pms = {'q' : q, 'count' : 100, 'lang' : 'en', 'result_type': 'recent'} 
auth = OAuth1(consumer_key, consumer_secret, access_token,access_token_secret)
#res = requests.get(url, params = pms, auth=auth)
#esta parte del código se utiliza realmente en la paginación

Sección de código que realiza la consulta **q** mediante la api de twitter utilizando los parámetros **pms**. Después, se crea una variable **auth** que corresponde a los datos exigidos por la API para conectarse que son *Las claves y los tokens de acceso.*
Por último se realiza la petición utilizando todos los datos necesarios por medio de una función de la librería requests **(request.get)** el resultado se vuelca en la variable **res**.

In [ ]:
#tweets = res.json()
#esta parte del código se utiliza realmente en la paginación

Si la conección es correcta se recuperarán varios documentos que se pueden convertir a json. En el formato json es en el cual extraeremos la información que nos importa.

## Carga de datos para más de 100 tweets (Paginación)

Como twitter te permite obtener hasta 100 tweets por llamada, si queremos recuperar más, necesitamos recordar los IDs de los tweets ya descargados para no repetir tweets en las siguientes llamadas. Esto es la paginación.

In [ ]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client[database_name]
collection = db[collection_name]

**MongoDB** *es una base de datos NoSQL ágil orientado a documentos, esto significa que en lugar de guardar los datos en tablas como se hace en las bbdd relacionales, MongoDB guarda estructuras de datos en documentos similares a JSON con un esquema dinámico.*
En nuestro caso importamos la librería para usar el cliente y creamos una base de datos y una colección a raíz de la base de datos. Ya cambiaré los nombres de **database_name** y **collection_name**.

In [ ]:
pages_counter = 0
number_of_pages = 100
while pages_counter < number_of_pages:
    pages_counter += 1
    res = requests.get(url, params = pms, auth=auth)
    print("Connection status: %s" % res.reason)
    tweets = res.json()
    ids = [i['id'] for i in tweets['statuses']]
    pms['max_id'] = min(ids) - 1
    collection.insert_many(tweets['statuses'])

El código para la paginación consiste en un bucle que se repetirá tantas veces como el número aproximado de tweets que queramos recuperar de twitter. Dentro del bucle se realiza la *consulta* (**requests.get**) y se guarda en la variable tweets con formato json (igual que se hacía antes de la paginación)
Después, para implementar lo que es realmente la paginación; 
- Se recolectan todos los **ids** de los tweets recogidos mediante el **get**.
- Se almacena en **i** todos los **ids** de todos los tweets.
- Se establece en **pms** como el *id mayor*, el *id mínimo* de **i** -1 (porque se incluiría y después se duplica)
- Por último se inserta en la base de datos (Con **insert_many**) todos los tweets que han sido recuperados en esta *ronda*